## Imports

In [ ]:
!pip install PyPDF2 --quiet
!pip install keybert --quiet

In [ ]:
import torch

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import json

In [ ]:
# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# import wordnet
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from gensim.models import Phrases
from gensim.corpora import Dictionary

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
from gensim.models import Phrases

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
from gensim.models import LdaModel, LdaMulticore

In [ ]:
import re

In [ ]:
from collections import defaultdict
import PyPDF2

In [ ]:
# !pip install flair


In [ ]:
from keybert import KeyBERT
from keybert.backend import BaseEmbedder
# from flair.embeddings import TransformerDocumentEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

# tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
# query_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import io

In [ ]:
!pip install faiss-gpu

In [ ]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
import faiss

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
import re

## Paragraph

In [ ]:
# pattern = r'fig(ure)?.?.?[0-9]+.[0-9]+'
# re.search( string="This is a figure 1.5 and it works", flags=re.IGNORECASE, pattern=pattern)

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

MODEL_STRING = "facebook/dpr-ctx_encoder-single-nq-base"

In [ ]:
class Paragraph:
    def __init__(self, book_id:int, id: int, text: str, embeddings: np.array, tagged_images: list=None):
        self.text = text
        self.embeddings = embeddings
        self.id = id
        self.book_id = book_id
    def get_json(self):
        dc = {
            "text": self.text,
            "id": self.id,
            "embeddings": self.embeddings
        }
        return json.dumps(dc)

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

In [ ]:
class EncoderModels():
    def __init__(self):
        ctx_model_string = "facebook/dpr-ctx_encoder-single-nq-base"
        query_model_string = "facebook/dpr-question_encoder-single-nq-base"

        DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.device = DEVICE

        self.ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(ctx_model_string)
        self.ctx_model = DPRContextEncoder.from_pretrained(ctx_model_string)
        self.ctx_model = self.ctx_model.to(DEVICE)

        self.query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(query_model_string)
        self.query_model = DPRQuestionEncoder.from_pretrained(query_model_string)
        self.query_model = self.query_model.to(DEVICE)

    def encode_context(self, paragraph: list[str],):
        num_paragraphs = len(paragraph)
        cloned_doc_embeddings = []
        batch_size = 32
        # print(paragraph)
        for itr in range(0, num_paragraphs, batch_size):
            s = itr
            e = min(itr+batch_size, num_paragraphs)
            tokenized = self.ctx_tokenizer(paragraph[s : e],
                                       padding=True, truncation=True,
                                       return_tensors="pt",
                                       return_attention_mask=True,
                                       add_special_tokens=True,
                                       max_length=512,
                                       pad_to_max_length=True
                                       ).to(self.device)
            input_ids = tokenized['input_ids']
            print(input_ids.shape)
            attention_mask = tokenized['attention_mask']
            print(attention_mask)
            with torch.no_grad():
                doc_embeddings = self.ctx_model(input_ids, attention_mask=attention_mask).pooler_output

            cloned_doc_embeddings.append(torch.clone(doc_embeddings).cpu().numpy())

            ##clean up
            torch.cuda.empty_cache()
            del input_ids

        return np.vstack(cloned_doc_embeddings)
    def encode_query(self, query):

        tokenized = self.query_tokenizer(query,
                                    padding=True, truncation=True,
                                    return_tensors="pt",
                                    return_attention_mask=True,
                                    add_special_tokens=True,
                                    max_length=512,
                                    pad_to_max_length=True
                                    ).to(self.device)
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        with torch.no_grad():
            doc_embeddings = self.query_model(input_ids, attention_mask=attention_mask).pooler_output
        embedding = torch.clone(doc_embeddings).cpu().numpy()

        ##clean up
        torch.cuda.empty_cache()
        del input_ids

        return embedding





In [ ]:
class EncoderModel():
    def __init__(self, model_string: str = MODEL_STRING, device: str= DEVICE):
        super().__init__()
        self.device = DEVICE
        self.model_string = model_string

        self.tokenizer = AutoTokenizer.from_pretrained(model_string)
        self.model = AutoModel.from_pretrained(model_string, resume_download=True)
        self.model = self.model.to(DEVICE)
        # self.tokenizer = self.tokenizer.to(self.device)


    def encode(self, paragraph: list[str],):
        num_paragraphs = len(paragraph)
        cloned_doc_embeddings = []
        batch_size = 32
        print(paragraph)
        for itr in range(0, num_paragraphs, batch_size):
            s = itr
            e = min(itr+batch_size, num_paragraphs)
            tokenized = self.tokenizer(paragraph[s : e],
                                       padding=True, truncation=True,
                                       return_tensors="pt",
                                       return_attention_mask=True,
                                       add_special_tokens=True,
                                       max_length=512,
                                       pad_to_max_length=True
                                       ).to(self.device)
            input_ids = tokenized['input_ids']
            print(input_ids.shape)
            attention_mask = tokenized['attention_mask']
            print(attention_mask)
            with torch.no_grad():
                doc_embeddings = self.model(input_ids, attention_mask=attention_mask).pooler_output

            cloned_doc_embeddings.append(torch.clone(doc_embeddings).cpu().numpy())

            ##clean up
            torch.cuda.empty_cache()
            del input_ids

        return np.vstack(cloned_doc_embeddings)


In [ ]:
# encoder = Encoder_Model()

In [ ]:
# emb = encoder.encode(["Its been a long night","tomowwor we win this game by hige margin"])

In [ ]:
# emb.shape

# PdfParagraphExtractor

In [ ]:
#pdf outline extration
class PdfParagraphExtractor:
    def __init__(self, book_path: str) -> None:
        # self.device = DEVICE
        self.book_path = book_path


        self.reader = PyPDF2.PdfReader(book_path)
        self.outlines = self.reader.outline
        self.ignore_outline_list = ['Contents', 'About Us', 'About', 'Preface', 'Glossary', 'Acknowledgement','References', 'Copyright', 'Cover', 'Table of Contents', 'Colophon', 'Index', 'Appendix', 'About the Authors']
        self.ignore_outline_pattern = r"(?i)\b(?:Contents?|About Us?|Preface|Glossary|Acknowledgement|References?|Copyright|Cover|Table of Contents|Colophon|Index|Appendix|About the Authors)\b"
        self.pdf_contents = dict()

        self.pages=self.reader.pages
        self.print_document_outline(self.reader, self.outlines)

        self.content_in_paragraphs = defaultdict(list)
        self.corpus = ""

        self.paragraphs = []
        self.paragraph_count = 0

        # self.encoder_model = encoder_model
        # self.doc_embeddings = None

        self.paragraph_generator()
        # self.compute_doc_embeddings()
        return

    def print_document_outline(self,reader, outlines, indent=0):
        for outline in outlines:
            if isinstance(outline, list):
                self.print_document_outline(reader, outline, indent + 1)
            else:
                title = outline.title
                # print(title)
                try:
                    page_number = reader.get_destination_page_number(outline)
                except:
                    page_number = -1


                self.pdf_contents[title] = page_number
                # print('  ' * indent + f"Title: {title}, Page: {page_number}")
    def print_mapping(self):
        print(self.pdf_contents)


    def paragraph_generator(self):
        self.pages = self.reader.pages
        last_page = 0
        pdf_Contents = list(self.pdf_contents.items())
        temp_count = 0
        for iterator in range(1,len(self.pdf_contents)):
            (outline,page_number) =  (pdf_Contents[iterator-1][0],pdf_Contents[iterator][1])
            if not re.match(self.ignore_outline_pattern, outline):
                for page in self.pages[last_page:page_number]:
                    # print(page.extract_text())
                    text = page.extract_text()
                    print('Text: ',text)
                    # print(text)
                    # print(text.splitlines())
                    # print(text.split('.\n'))
                    paragraphs_arr = text.split('.\n')

                    for paragraph in paragraphs_arr:
                        paragraph = paragraph.replace('\n', '').replace('\t', '')
                        if(paragraph!='' and paragraph!=None and len(paragraph.split())>0):
                            self.paragraphs.append(paragraph)

                            temp_count+=1

                            self.content_in_paragraphs[outline].append(self.paragraph_count)
                            self.paragraph_count += 1
            last_page = page_number
    def get_paragraph_list(self):
        return self.paragraphs


pdf_paths =[ "/content/Anderson OS 1.pdf", "/content/Anderson OS 2.pdf", "/content/Anderson OS 3.pdf"]

# print("Document Outline:")
# outlinegen = PdfParagraphExtractor(pdf_path)
# outlinegen.pdf_reader(pdf_path)
# outlinegen.outline_generator()
# outlinegen.print_document_outline(outlinegen.reader,outlinegen.outlines)
# # outlinegen.print_mapping()
# outlinegen.paragraph_generator()

In [ ]:
class PdfParagraphExtractor2:
    def __init__(self, book_path: str):
        self.book_path = book_path

        self.reader = PyPDF2.PdfReader(book_path)

        self.paragraphs = []
        self.paragraph_count = 0

        self.extract_paragraphs()
    def extract_paragraphs(self):
        for page in self.reader.pages:
            text = page.extract_text()
            paragraphs = text.split('.\n')
            for paragraph in paragraphs:
                paragraph = paragraph.replace('\n', '').replace('\t', '')
                if(paragraph!='' and paragraph!=None and len(paragraph.split())>0):
                    self.paragraphs.append(paragraph)
        self.paragraph_count = len(self.paragraphs)



In [ ]:
# outlinegen.document_embeddings.shape

In [ ]:
# encoder_model = EncoderModel()
# outlinegen = PdfParagraphExtractor(pdf_path, encoder_model)

# Key Bert

In [ ]:
class KeyBertModel():
    def __init__(self, docs, doc_embeddings):
        self.kw_model = KeyBERT()
        self.top_eles = 5

        self.key_map = dict()
        self.score_map = dict()
        self.key_words_per_docs = None
        self.key_to_ids = dict()

        self.extract(docs, doc_embeddings)

        # required objects
        self.hyperedge_dictionary = self.key_to_ids
        self.hyperedge_embeddings = self.key_map

    def extract(self, docs, doc_embeddings):
        self.key_words_per_docs = self.kw_model.extract_keywords(docs, keyphrase_ngram_range=(1, 2), use_mmr=True, diversity=0.4, top_n=self.top_eles)
        self.set_keyword_map(doc_embeddings)
        return

    def set_keyword_map(self,doc_embeddings, threshold=0.3):
        for idx, key_list in enumerate(self.key_words_per_docs):
            for item in key_list:
                key, score = item
                if score>=threshold:
                    if key in self.key_map.keys():
                        self.key_to_ids[key].append(idx)
                        self.key_map[key]+= doc_embeddings[idx]
                        self.score_map[key]+= score
                    else:
                        self.key_to_ids[key] = [idx]
                        self.key_map[key] = doc_embeddings[idx]
                        self.score_map[key] = score
        for key in list(self.key_map):
            self.key_map[key] = self.key_map[key] / self.score_map[key]
        return

    def get_hypergraph(self):
        key_to_keyid = {}
        keyid_to_key = {}
        n = len(self.hyperedge_dictionary)
        itr = 0
        for key in list(self.hyperedge_dictionary.keys()):
            key_to_keyid[key] = itr
            keyid_to_key[itr] = key
            itr+=1
        return (self.hyperedge_dictionary, self.hyperedge_embeddings, key_to_keyid, keyid_to_key)







In [ ]:
# kw_model = KeyBertModel()

In [ ]:
# kw_model.extract(outlinegen.paragraphs, outlinegen.document_embeddings)

In [ ]:
# kw_model.key_words_per_docs

In [ ]:
# kw_model.get_keyword_map(outlinegen.document_embeddings)

# LDA

In [ ]:
class LDA():
    # while initializing the object, the input is an array of paragraph objects
    # now list of paragraphs
    def __init__(self, docs, doc_embeddings) -> None:
        ##objects that will be used for text cleaning
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.stopWords = stopwords.words('english')
        self.punctuations = set(string.punctuation)
        self.lemma = WordNetLemmatizer()

        ##ldamodel
        self.lda_model = None
        self.is_num_topic_set = False
        self.num_topics = 0

        ##corpus
        self.paragraphs = docs
        self.doc_embeddings = doc_embeddings
        self.clean_corpus = [self.apply_cleaning_for_lda(doc) for doc in self.paragraphs]
        self.clean_corpus = self.add_bigrams(self.clean_corpus)

        ##bag of words representation
        self.dictionary = Dictionary(self.clean_corpus)
        self.bow_representation_of_corpus = [self.dictionary.doc2bow(doc) for doc in self.clean_corpus]

        ## topic document mappings
        self.hyperedge_dictionary = {}
        self.hyperedge_embeddings = {}
        self.hyperedge_probabilities = {}

        ##compute everything
        self.find_optimal_model()
        self.compute_topic_document_mapping()
        self.compute_hypergraph_embeddings()

        ##array for mapping
        self.arr_topics = list(self.hyperedge_embeddings.keys())
        self.arr_embeddings = np.vstack(list(self.hyperedge_embeddings.values()))


    def tokenize(self, x):
        return self.tokenizer.tokenize(x)

    def clean_string(self,x):
        return re.sub(r'[,\.!?]', '', x).replace('\n', '').replace('\t', '')

    def lower(self,x):
        return x.lower()

    def remove_stopwords(self,x):
        x = [word for word in x if word not in self.stopWords]
        return x

    def remove_punctuations(self, x):
        x = [word for word in x if word not in self.punctuations]
        return x

    def lemmatize(self, x):
        x = [self.lemma.lemmatize(word) for word in x]
        return x

    def remove_numbers(self, x):
        return [token for token in x if not token.isnumeric()]

    def remove_single_characters(self, x):
        return [token for token in x if len(token)>1]


    def add_bigrams(self, x):
        bigram = Phrases(x, min_count=20)
        for idx in range(len(x)):
            for token in bigram[x[idx]]:
                if '_' in token:
                    # Token is a bigram, add to document.
                    x[idx].append(token)
        return x

    def train_lda_model(self, num_topics,chunksize=2000, passes=40, iterations=1000, eval_every=None):
        temp = self.dictionary[0]
        id2word = self.dictionary.id2token

        model = LdaModel(
            corpus=self.bow_representation_of_corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha='auto',
            eta='auto',
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=eval_every
        )

        return model

    def get_coherence_score(self, model):
        coherence_model = CoherenceModel(model=model, corpus=self.bow_representation_of_corpus, coherence='u_mass')
        return coherence_model.get_coherence()

    def find_optimal_model(self):
        # k_min = len(self.clean_corpus)//10
        k_max = len(self.clean_corpus)//4

        self.lda_model = self.train_lda_model(num_topics=k_max)
        self.is_num_topic_set = True
        self.num_topics = k_max

        return

        # step = (k_max-k_min)//10
        # if step==0:
        #     step = 1

        # coh_score = -1e9
        # num_topics = k_min
        # itr_num_topics = num_topics

        # while itr_num_topics<=k_max:
        #     model = self.train_lda_model(num_topics=itr_num_topics)
        #     score = self.get_coherence_score(model)
        #     if(score>coh_score):
        #         score = coh_score
        #         num_topics = itr_num_topics
        #         self.lda_model = model
        #     itr_num_topics+=step
        # self.num_topics = num_topics
        # self.is_num_topic_set = True
        # return

    def get_topics(self):
        return self.lda_model.get_topics()

    def get_vocabulary(self):
        return self.dictionary.id2token


    def compute_topic_document_mapping(self, threshold=0.4):
        topic_dic = defaultdict(list)
        hg_emb = {}
        hg_probs = defaultdict(float)

        n = self.num_topics

        for doc_id, doc_bow in enumerate(self.bow_representation_of_corpus):
            topic_distribution = self.lda_model.get_document_topics(doc_bow, minimum_probability=None)

            for topic_id, prob in topic_distribution:
                if prob>=threshold:
                    topic_dic[topic_id].append((doc_id, prob))

                    hg_probs[topic_id]+=prob

                    emb = prob * self.doc_embeddings[doc_id]
                    if topic_id in hg_emb.keys():
                        hg_emb[ topic_id ] += emb
                    else:
                        hg_emb[ topic_id ]=emb

        self.hyperedge_dictionary = topic_dic
        self.hyperedge_embeddings = hg_emb
        self.hyperedge_probabilities = hg_probs
        return

    def compute_hypergraph_embeddings(self):
        for topic_id in range(self.num_topics):
            if topic_id in self.hyperedge_embeddings.keys():
                self.hyperedge_embeddings[topic_id] = self.hyperedge_embeddings[topic_id]/self.hyperedge_probabilities[topic_id]
        return


    def apply_cleaning_for_lda(self, x):
        x = self.clean_string(x)
        x = self.lower(x)
        x = self.tokenize(x) ##Now the returned element is a list of words
        x = self.remove_stopwords(x)
        x = self.remove_punctuations(x)
        x = self.remove_single_characters(x)
        x = self.remove_numbers(x)
        x = self.lemmatize(x)
        return x
    def get_hypergraph(self):
        key_to_keyid = {}
        keyid_to_key = {}
        n = len(self.hyperedge_dictionary)
        itr = 0
        for key in list(self.hyperedge_dictionary.keys()):
            key_to_keyid[key] = itr
            keyid_to_key[itr] = key
            itr+=1
        return (self.hyperedge_dictionary, self.hyperedge_embeddings, key_to_keyid, keyid_to_key)

# lda = LDA(list(outlinegen.paragraphs.values()))


# Faiss

In [ ]:
class Faiss:
    def __init__(self,doc_embeddings,model,top_k=15,dimension=768):
        # doc embeddings is an array of vectors of shape (1,768)
        self.doc_embeddings = np.vstack(doc_embeddings)
        self.top_k=top_k
        self.dimension=dimension
        res = faiss.StandardGpuResources()
        flat_config = faiss.GpuIndexFlatConfig()

        faiss.normalize_L2(self.doc_embeddings)
        self.index=faiss.GpuIndexFlatL2(res,self.dimension,flat_config)

        self.query_model=model
        self.create_index()
    def create_index(self):
        # Add your vector data to the index
        self.index.add(self.doc_embeddings)
        return self.index.ntotal
    def serve_query(self,question):
        query_embedding = self.query_model.encode_query(question)

        faiss.normalize_L2(query_embedding)

        D, I = self.index.search(query_embedding, self.top_k)
        return D,I


In [ ]:
# from keybert import KeyBERT
# from flair.embeddings import TransformerDocumentEmbeddings

# roberta = TransformerDocumentEmbeddings('facebook/dpr-ctx_encoder-single-nq-base')
# kw_model = KeyBERT(model=roberta)

# def generate_keys(HyperGraph_Map, paragraph_id, paragraphs, keyBERT=kw_model, top_eles=3):
#     doc = paragraphs[paragraph_id].text

#     # Extract key phrases of different lengths in a single pass
#     key_phrases = []
#     for n in range(2, 3):
#         key_phrases.extend(keyBERT.extract_keywords(doc, keyphrase_ngram_range=(n, n), use_mmr=True, diversity=0.7, top_n=top_eles))

#     for key, prob in key_phrases:
#         embeddings = keyBERT.extract_embeddings(doc, candidates=[key], keyphrase_ngram_range=(len(key.split()), len(key.split())), stop_words=None)
#         # print(len(embeddings))
#         if embeddings:
#             key_list = {}
#             if key not in HyperGraph_Map:
#                 key_list = {"keyword_embeddings": embeddings[1] * prob, "prob_sum": prob}
#             # key_list["keyword_embeddings"] += (embeddings * torch.tensor(prob))
#             else:
#                 key_list["keyword_embeddings"] = HyperGraph_Map[key]["keyword_embeddings"]+ embeddings[1] * prob
#                 key_list["prob_sum"] = HyperGraph_Map[key]["prob_sum"] + prob
#             HyperGraph_Map[key] = key_list

#     return

# count=0
# for content in outlinegen.pdf_contents:
#     # print(content)
#     paragraph_ids = outlinegen.content_in_paragraphs[content]
#     # print(len(paragraphs))
#     # print(paragraphs)crea
#     for paragraph_id in paragraph_ids:
#         if len(outlinegen.paragraphs[paragraph_id].text)!=0:
#             count+=1
#             # print(paragraph.embeddings)
#             generate_keys(outlinegen.HyperGraph_Map,paragraph_id,outlinegen.paragraphs)

# Hypergraph Class

In [ ]:
class Hypergraph():
    # all the hypergraph related mapping and its index
    def __init__(self, name, key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key, encoder_models):
        self.name = name
        self.key_to_docids = key_to_docids
        self.key_to_embeddings = key_to_embeddings
        self.key_to_keyid = key_to_keyid
        self.keyid_to_key = keyid_to_key

        self.vector_db = Faiss(list(self.key_to_embeddings.values()), encoder_models)

    def query_hypergraph(self, question):
        D, I = self.vector_db.serve_query(question)
        key_ids = I[0]
        ans_arr = []
        for key_id in key_ids:
            key = self.keyid_to_key[key_id]
            docids = self.key_to_docids[key]
            ans_arr.extend(docids)
        return ans_arr




# Reader

In [ ]:
!wget https://dl.fbaipublicfiles.com/FiD/pretrained_models/nq_reader_large.tar.gz

--2024-08-31 18:27:16--  https://dl.fbaipublicfiles.com/FiD/pretrained_models/nq_reader_large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.96, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2732989981 (2.5G) [application/gzip]
Saving to: ‘nq_reader_large.tar.gz’

nq_reader_large.tar   3%[                    ]  89.99M  33.1MB/s               ^C


In [ ]:
!tar -xvzf /content/nq_reader_large.tar.gz

nq_reader_large/
nq_reader_large/config.json
nq_reader_large/pytorch_model.bin

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
import torch

In [ ]:
from transformers import T5Tokenizer

In [ ]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np

class FiDT5(transformers.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FiDT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        if input_ids != None:
            # inputs might have already be resized in the generate method
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1)
        if attention_mask != None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length):
        self.encoder.n_passages = input_ids.size(1)
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    """
    Encoder Wrapper for T5 Wrapper to obtain a Fusion-in-Decoder model.
    """
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()

        self.encoder = encoder
        self.embed_tokens = encoder.embed_tokens
        self.main_input_name=encoder.main_input_name
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

    def forward(self, input_ids=None, attention_mask=None, **kwargs,):
        # total_length = n_passages * passage_length
        bsz, total_length = input_ids.shape
        passage_length = total_length // self.n_passages
        input_ids = input_ids.view(bsz*self.n_passages, passage_length)
        attention_mask = attention_mask.view(bsz*self.n_passages, passage_length)
        outputs = self.encoder(input_ids, attention_mask, **kwargs)
        outputs = (outputs[0].view(bsz, self.n_passages*passage_length, -1), ) + outputs[1:]
        return outputs

class CheckpointWrapper(torch.nn.Module):
    """
    Wrapper replacing None outputs by empty tensors, which allows the use of
    checkpointing.
    """
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    """
    Wrap each block of the encoder to enable checkpointing.
    """
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

def cross_attention_forward(
        self,
        input,
        mask=None,
        kv=None,
        position_bias=None,
        past_key_value_state=None,
        head_mask=None,
        query_length=None,
        use_cache=False,
        output_attentions=False,
    ):
    """
    This only works for computing cross attention over the input
    """
    assert(kv != None)
    assert(head_mask == None)
    assert(position_bias != None or self.has_relative_attention_bias)

    bsz, qlen, dim = input.size()
    n_heads, d_heads = self.n_heads, self.d_kv
    klen = kv.size(1)

    q = self.q(input).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    if past_key_value_state == None:
        k = self.k(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
        v = self.v(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    else:
        k, v = past_key_value_state

    scores = torch.einsum("bnqd,bnkd->bnqk", q, k)

    if mask is not None:
       scores += mask

    if position_bias is None:
        position_bias = self.compute_bias(qlen, klen)
    scores += position_bias

    if self.score_storage is None:
        self.score_storage = scores

    attn = F.softmax(scores.float(), dim=-1).type_as(scores)
    attn = F.dropout(attn, p=self.dropout, training=self.training)

    output = torch.matmul(attn, v)
    output = output.transpose(1, 2).contiguous().view(bsz, -1, self.inner_dim)
    output = self.o(output)

    if use_cache:
        output = (output,) + ((k, v),)
    else:
        output = (output,) + (None,)

    if output_attentions:
        output = output + (attn,)

    if self.has_relative_attention_bias:
        output = output + (position_bias,)

    return output

In [ ]:
doc1='''
Simplified control: Docker Compose allows you to define and manage multi-container applications in a single YAML file. This simplifies the complex task of orchestrating and coordinating various services, making it easier to manage and replicate your application environment.

Efficient collaboration: Docker Compose configuration files are easy to share, facilitating collaboration among developers, operations teams, and other stakeholders. This collaborative approach leads to smoother workflows, faster issue resolution, and increased overall efficiency.

Rapid application development: Compose caches the configuration used to create a container. When you restart a service that has not changed, Compose re-uses the existing containers. Re-using containers means that you can make changes to your environment very quickly.
'''

In [ ]:
doc2='''
Portability across environments: Compose supports variables in the Compose file. You can use these variables to customize your composition for different environments, or different users.

Extensive community and support: Docker Compose benefits from a vibrant and active community, which means abundant resources, tutorials, and support. This community-driven ecosystem contributes to the continuous improvement of Docker Compose and helps users troubleshoot issues effectively.
'''

In [ ]:
doc3='''
Development environments
When you're developing software, the ability to run an application in an isolated environment and interact with it is crucial. The Compose command line tool can be used to create the environment and interact with it.

The Compose file provides a way to document and configure all of the application's service dependencies (databases, queues, caches, web service APIs, etc). Using the Compose command line tool you can create and start one or more containers for each dependency with a single command (docker compose up).

Together, these features provide a convenient way for you to get started on a project. Compose can reduce a multi-page "developer getting started guide" to a single machine-readable Compose file and a few commands.
'''

In [ ]:
doc4='''
Docker is a software platform that allows you to build, test, and deploy applications quickly. Docker packages software into standardized units called containers that have everything the software needs to run including libraries, system tools, code, and runtime. Using Docker, you can quickly deploy and scale applications into any environment and know your code will run.
Running Docker on AWS provides developers and admins a highly reliable, low-cost way to build, ship, and run distributed applications at any scale.
'''

In [ ]:
doc5='''
'''

In [ ]:
docs=[doc1,doc2,doc3,doc4]

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
# model_class = FiDT5
# model = FiDT5.from_pretrained("/content/nq_reader_base")
# model = model.to(device)

In [ ]:
# print(FiDT5)

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base', return_dict=False)

In [ ]:
question="Explain Docker"

In [ ]:
def append_question(question,docs):
    return [question + " " + t for t in docs]
text_passages = [append_question(question,docs) for passege in docs]

In [ ]:
def encode_passages(batch_text_passages, tokenizer, max_length):
    passage_ids, passage_masks = [], []
    for k, text_passages in enumerate(batch_text_passages):
        p = tokenizer(
            text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        passage_ids.append(p['input_ids'][None])
        passage_masks.append(p['attention_mask'][None])

    passage_ids = torch.cat(passage_ids, dim=0)
    passage_masks = torch.cat(passage_masks, dim=0)
    return passage_ids, passage_masks.bool()
# context_ids,context_mask=encode_passages(text_passages,tokenizer,2000)

In [ ]:
def encode_passages(batch_text_passages, tokenizer, max_length):
    passage_ids, passage_masks = [], []
    for k, text_passages in enumerate(batch_text_passages):
        p = tokenizer(
            text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        passage_ids.append(p['input_ids'][None])
        passage_masks.append(p['attention_mask'][None])

    passage_ids = torch.cat(passage_ids, dim=0)
    passage_masks = torch.cat(passage_masks, dim=0)
    return passage_ids, passage_masks.bool()

def append_question(question,docs):
    return [question + " " + t for t in docs]

def generate(docs, question):
    text_passages = [append_question(question,docs) for passege in docs]
    context_ids,context_mask=encode_passages(text_passages,tokenizer,1000)
    ans=None
    with torch.no_grad():
        outputs = model.generate(
                input_ids=context_ids.to(device),
                attention_mask=context_mask.to(device),
                max_length=100)
        for k, o in enumerate(outputs):
            ans = tokenizer.decode(o, skip_special_tokens=True)
    return ans



In [ ]:
# outputs = model.generate(
#                 input_ids=context_ids.to(device),
#                 attention_mask=context_mask.to(device),
#                 max_length=100)

In [ ]:
# ans=None
# with torch.no_grad():
#     outputs = model.generate(
#             input_ids=context_ids.to(device),
#             attention_mask=context_mask.to(device),
#             max_length=100)
#     for k, o in enumerate(outputs):
#         ans = tokenizer.decode(o, skip_special_tokens=True)

In [ ]:
# for k, o in enumerate(outputs):
#     ans = tokenizer.decode(o, skip_special_tokens=True)

In [ ]:
# ans

In [ ]:
# # Load config from JSON file
# model_class=T5ForConditionalGeneration
# config = AutoConfig.from_pretrained("/content/nq_reader_base/config.json")

# # Load model weights from binary file
# model = model_class.from_pretrained("/content/nq_reader_base/pytorch_model.bin", config=config)
# model=model.to(device)

In [ ]:
# from transformers import AutoTokenizer, LongformerForQuestionAnswering
# import torch

# tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
# model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")



In [ ]:
# question = "What is Docker compose file?"
# text= " ".join(docs)
# encoding = tokenizer(question, text, return_tensors="pt")
# input_ids = encoding["input_ids"]

# # default is local attention everywhere
# # the forward method will automatically set global attention on question tokens
# attention_mask = encoding["attention_mask"]

# outputs = model(input_ids, attention_mask=attention_mask)
# start_logits = outputs.start_logits
# end_logits = outputs.end_logits
# all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

# answer_tokens = all_tokens[torch.argmax(start_logits) : torch.argmax(end_logits) + 1]
# answer = tokenizer.decode(
#     tokenizer.convert_tokens_to_ids(answer_tokens)
# )  # remove space prepending space token

In [ ]:
# answer

In [ ]:
# ans

In [ ]:
# for ans_token in answer_tokens:
#   ans

# Book

In [ ]:
class Models():
    def __init__(self):
        self.encoder_models = EncoderModels()
        # self.decoder_model = FiDT5.from_pretrained("/content/nq_reader_large")
        # self.decoder_tokenizer = T5Tokenizer.from_pretrained('t5-base', return_dict=False)
        # self.decoder_model = self.decoder_model.to(device)


In [ ]:
class Book():
    def __init__(self, book_paths, models):
        self.book_paths = book_paths
        self.models = models

        #some global identifiers
        self.global_paragraphs = None
        self.global_doc_embeddings = None
        self.pobj = None
        self.num_paragraphs = 0

        #read book
        self.books = [PdfParagraphExtractor2(book_path) for book_path in book_paths]
        print(self.books[0].paragraphs)
        self.bookid_to_pid = defaultdict(list)
        self.pid_to_bookid = defaultdict(int)
        self.set_global_info()


    def set_global_info(self):
        all_ps = []
        all_objs = []
        all_doc_embeddings = []

        id = 0
        for bookid, book in enumerate(self.books):
            doc_embeddings_by_book = self.compute_doc_embeddings(book.paragraphs)

            for pid, paragraph in enumerate(book.paragraphs):
                all_ps.append(paragraph)
                all_doc_embeddings.append(doc_embeddings_by_book[pid])

                self.bookid_to_pid[bookid].append(id)
                self.pid_to_bookid[id] = bookid
                id+=1

        self.num_paragraphs = len(all_ps)
        self.global_paragraphs = all_ps
        self.global_doc_embeddings = all_doc_embeddings

        for id in range(self.num_paragraphs):
            all_objs.append(Paragraph(self.pid_to_bookid[id], id, all_ps[id], all_doc_embeddings[id]))
        self.pobj = all_objs

        return

    def compute_doc_embeddings(self, para_list):
        doc_embeddings = self.models.encoder_models.encode_context(para_list)
        r, c = doc_embeddings.shape
        doc_embeddings = [doc_emb[:].reshape(1,-1) for doc_emb in doc_embeddings]
        return doc_embeddings

    def get_docs_from_docids(self, docids):
        ans = []
        for docid in docids:
            ans.append(self.global_paragraphs[docid])
        return " ".join(ans)

    def generate_normal_hg(self):
        n = len(self.global_paragraphs)
        key_to_docids = {}
        key_to_keyid = {}
        keyid_to_key = {}
        key_to_embeddings = {}
        for i in range(n):
            key_to_docids[i] = [i]
            key_to_keyid[i] = i
            keyid_to_key[i] = i
            key_to_embeddings[i] = self.global_doc_embeddings[i]

        normal = Hypergraph('normal', key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key, self.models.encoder_models)
        return normal




In [ ]:
def create_hg_from_model(name, model, encoder_models):
    key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key = model.get_hypergraph()
    hg = Hypergraph(name, key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key, model.encoder_models)
    return hg

In [ ]:
book_paths = ['/content/test2.pdf', '/content/test.pdf']

In [ ]:
models = Models()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dens

In [ ]:
books = Book(book_paths, models)

['Health Insurance Coverage Status and Type by Geography: 2021 and 2022American Community Survey BriefsACSBR-015Issued September 2023Douglas Conway and Breauna BranchINTRODUCTIONDemographic shifts as well as economic and govern-ment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States during that time', '1 Public policy changes included the renewal of the Public Health Emergency, which allowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision', '2 The American Rescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people', '3In addition to national policies, individual states and the District of Columbia can affect health insurance coverage by making Marketplace or Medicaid more accessible and affordable. This variation may b

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2783: UserWarning: Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.
  warnings.warn("Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.")


torch.Size([32, 423])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
torch.Size([32, 512])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
torch.Size([19, 512])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
torch.Size([27, 512])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 

In [ ]:
hg = books.generate_normal_hg()

In [ ]:
question = 'What is scope of work for MPDA?'

In [ ]:
extracted_text = books.get_docs_from_docids(hg.query_hypergraph(question=question))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2783: UserWarning: Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.
  warnings.warn("Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.")


In [ ]:
extracted_text

"gov), or coverage through TRICARE. Public insurance coverage includes federal programs (such as Medicare, Medicaid, and the Children’s Health Insurance Program or CHIP), individual state health plans, and CHAMPVA (Civilian Health and Medical Program at the Department of Veterans Affairs), as well as care provided by the Department of Veterans Affairs. In the ACS, people are considered insured if they were covered by any of these types of health insurance at time of interview. People are considered uninsured if they were not covered by any of these types of health insurance at time of interview or if they only had coverage through the Indian Health Service (IHS), as IHS coverage is not considered comprehensive 2 U.S. Census BureauWHAT IS HEALTH INSURANCE COVERAGE?This brief presents state-level estimates of health insurance coverage using data from the American Community Survey (ACS). The  U.S. Census Bureau conducts the ACS throughout the year; the survey asks respondents to report th

In [ ]:
prompt = f'Given the context: {extracted_text} /n Answer the question {question}'

In [ ]:
prompt

"Given the context: gov), or coverage through TRICARE. Public insurance coverage includes federal programs (such as Medicare, Medicaid, and the Children’s Health Insurance Program or CHIP), individual state health plans, and CHAMPVA (Civilian Health and Medical Program at the Department of Veterans Affairs), as well as care provided by the Department of Veterans Affairs. In the ACS, people are considered insured if they were covered by any of these types of health insurance at time of interview. People are considered uninsured if they were not covered by any of these types of health insurance at time of interview or if they only had coverage through the Indian Health Service (IHS), as IHS coverage is not considered comprehensive 2 U.S. Census BureauWHAT IS HEALTH INSURANCE COVERAGE?This brief presents state-level estimates of health insurance coverage using data from the American Community Survey (ACS). The  U.S. Census Bureau conducts the ACS throughout the year; the survey asks respo

In [ ]:
!pip install langchain
!pip install ctransformers
!pip install ctransformers[cuda]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
from langchain.llms import CTransformers

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# os.environ['XDG_CACHE_HOME'] = '/content/LLM_data/model/cache'
# model_id = 'TheBloke/Orca-2-13B-GGUF'

# config = {'temperature':0.00,'max_new_tokens': 512, 'context_length':4000,'gpu_layers':50,'repetition_penalty':1 }
# llm = CTransformers(model=model_id,
#                     model_type="llama",
#                     gpu_layers=50,
#                     device = 0,
#                     config=config,
#                     callbacks=[StreamingStdOutCallbackHandler()],
#                    )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

orca-2-13b.Q2_K.gguf:   0%|          | 0.00/5.43G [00:00<?, ?B/s]

In [ ]:
# llm(prompt)

<ipython-input-70-5b4e25c8175c>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm(prompt)


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
groq_api_key="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"

# st.title("Objectbox VectorstoreDB With Llama3 Demo")

llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

In [ ]:
response = llm.invoke(prompt)

In [ ]:
response.content

"The scope of work for MPDA entails the implementation of a new API architecture aimed at enhancing integration and interoperability capabilities across both internal applications and external strategic partners. The project involves the deployment of the new architecture, coupled with product implementation, migration from the current ASAB 1.0 to the new ASAB 2.0, and post-project support.\n\nThe key aspects of the project scope include:\n\n1. Adoption of a proven and industry-standard solution based on Open-API principles, ensuring comprehensive documentation, enhanced security, and real-time integration capabilities.\n2. Focus on improving visibility of existing APIs, reducing time-to-market, and simplifying complex operations and troubleshooting processes.\n3. Implementation of the API Management solution and seamless integration with existing application systems, including migration to the new ASAB 2.0.\n4. Design and development of a robust and scalable API architecture meeting M

In [ ]:
len(books.global_doc_embeddings)

703

In [ ]:
lda_hg = LDA(books.global_paragraphs, books.global_doc_embeddings)

In [ ]:
key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key = lda_hg.get_hypergraph()

In [ ]:
keyid_to_key

{0: 47,
 1: 7,
 2: 159,
 3: 61,
 4: 90,
 5: 11,
 6: 78,
 7: 118,
 8: 18,
 9: 31,
 10: 50,
 11: 151,
 12: 75,
 13: 160,
 14: 79,
 15: 115,
 16: 105,
 17: 8,
 18: 16,
 19: 94,
 20: 153,
 21: 57,
 22: 164,
 23: 9,
 24: 103,
 25: 45,
 26: 69,
 27: 24,
 28: 55,
 29: 0,
 30: 172,
 31: 76,
 32: 29,
 33: 170,
 34: 65,
 35: 81,
 36: 41,
 37: 110,
 38: 13,
 39: 48,
 40: 119,
 41: 166,
 42: 111,
 43: 35,
 44: 67,
 45: 104,
 46: 173,
 47: 125,
 48: 1,
 49: 91,
 50: 88,
 51: 96,
 52: 64,
 53: 138,
 54: 116,
 55: 121,
 56: 95,
 57: 168,
 58: 107,
 59: 109,
 60: 100,
 61: 22,
 62: 62,
 63: 32,
 64: 114,
 65: 39,
 66: 144,
 67: 28,
 68: 30,
 69: 89,
 70: 99,
 71: 21,
 72: 136,
 73: 19,
 74: 44,
 75: 131,
 76: 54,
 77: 148,
 78: 120,
 79: 140,
 80: 155,
 81: 82,
 82: 98,
 83: 26,
 84: 137,
 85: 85,
 86: 4,
 87: 133,
 88: 42,
 89: 132,
 90: 23,
 91: 60,
 92: 147,
 93: 158,
 94: 112,
 95: 154,
 96: 135,
 97: 72,
 98: 86,
 99: 128,
 100: 68,
 101: 66,
 102: 93,
 103: 6,
 104: 77,
 105: 83,
 106: 150,
 107

In [ ]:
key_to_docids

defaultdict(list,
            {122: [(0, 0.9673714), (290, 0.9851039)],
             107: [(1, 0.9827809),
              (115, 0.981312),
              (143, 0.971629),
              (180, 0.40874648),
              (227, 0.77317286),
              (297, 0.7530678),
              (469, 0.49594155),
              (474, 0.6074802),
              (480, 0.44561306),
              (508, 0.88884664),
              (517, 0.89679456),
              (519, 0.6268717),
              (656, 0.8114617),
              (670, 0.8013136)],
             24: [(2, 0.98536015),
              (4, 0.4376217),
              (166, 0.9685581),
              (386, 0.516569),
              (514, 0.9816912),
              (516, 0.522803),
              (518, 0.5341653),
              (520, 0.47671738)],
             70: [(3, 0.97333735),
              (36, 0.98206437),
              (114, 0.987272),
              (118, 0.9848532),
              (152, 0.97486585),
              (432, 0.8637952),
              (534, 

In [ ]:
def analyse_hypergraph(key_to_ids):
    mx = -1e9
    mn = 1e9
    mean = 0
    median = 0
    arr_of_len = []
    n = len(key_to_ids)

    for i, key in enumerate(list(key_to_ids.keys())):
        size = len(key_to_ids[key])
        arr_of_len.append(size)

        mx = max(mx, size)
        mn = min(size, mn)
        mean+=size
        print(key_to_ids[key])
    mean/=n
    arr_of_len.sort()
    print("Array of lenghts: ",arr_of_len)
    if(n%2==0):
        median = (arr_of_len[n//2]+arr_of_len[(n//2)-1])/2
    else:
        median = arr_of_len[n//2]
    print("Mean: ",mean)
    print("Max: ",mx)
    print("Min: ",mn)
    print("Median: ", median)

In [ ]:
analyse_hypergraph(key_to_docids)

[(0, 0.9673714), (290, 0.9851039)]
[(1, 0.9827809), (115, 0.981312), (143, 0.971629), (180, 0.40874648), (227, 0.77317286), (297, 0.7530678), (469, 0.49594155), (474, 0.6074802), (480, 0.44561306), (508, 0.88884664), (517, 0.89679456), (519, 0.6268717), (656, 0.8114617), (670, 0.8013136)]
[(2, 0.98536015), (4, 0.4376217), (166, 0.9685581), (386, 0.516569), (514, 0.9816912), (516, 0.522803), (518, 0.5341653), (520, 0.47671738)]
[(3, 0.97333735), (36, 0.98206437), (114, 0.987272), (118, 0.9848532), (152, 0.97486585), (432, 0.8637952), (534, 0.97801715)]
[(4, 0.50765073), (216, 0.44801924), (219, 0.6894008), (221, 0.9834661), (278, 0.4948008), (311, 0.9536941), (340, 0.7436217), (351, 0.7296594), (358, 0.4537959), (480, 0.42818072), (488, 0.59062153), (502, 0.74097085), (506, 0.4307301), (507, 0.6623552)]
[(5, 0.98726845), (141, 0.99002105), (220, 0.8460758)]
[(6, 0.9509562)]
[(7, 0.7001358), (11, 0.9934001)]
[(8, 0.6231549), (666, 0.98484546)]
[(9, 0.7351329), (94, 0.9824203), (254, 0.97

In [ ]:
analyse_hypergraph(key_to_docids)

[(0, 0.9661555), (515, 0.96954954), (536, 0.929472), (537, 0.98528725)]
[(1, 0.9821107), (404, 0.3740168), (590, 0.3252819), (591, 0.9731441), (638, 0.80121064), (695, 0.9834137), (698, 0.9345468)]
[(2, 0.9847939), (667, 0.9542539)]
[(3, 0.9723301), (9, 0.33764926), (28, 0.43996596), (74, 0.9871568), (99, 0.98245645), (139, 0.77188915), (153, 0.26409817)]
[(4, 0.5358544), (182, 0.92034334), (265, 0.978273), (396, 0.2875619), (438, 0.7408674), (615, 0.9461633), (621, 0.23423994), (624, 0.97774214), (701, 0.60657895)]
[(4, 0.40724796), (10, 0.97971845), (103, 0.98715436), (504, 0.81307036), (520, 0.44600525)]
[(5, 0.9867802), (380, 0.2566426)]
[(6, 0.94915515), (55, 0.5161626), (57, 0.98016083), (379, 0.63621193)]
[(7, 0.9461487), (69, 0.97391015), (231, 0.949154), (354, 0.9884567)]
[(8, 0.9564618), (148, 0.9896418), (659, 0.2841795), (673, 0.9891481), (674, 0.30999336), (676, 0.3542515), (677, 0.3606524), (678, 0.9834176), (680, 0.28733847), (681, 0.36756364), (682, 0.4502029), (684, 0.

In [ ]:
def print_if_above_threshold(docs, key_to_ids, threshold=7):
    for key in list(key_to_ids.keys()):
        if len(key_to_ids[key])>=threshold:
            print(f"Topic {key} is above threhsold, it contains the following documents: ")
            map = key_to_ids[key]
            for entity in map:
                id, prob = entity
                print("Probability: ",prob)
                print("Doc: ",docs[id])




In [ ]:
def print_retrieved(docs, key_to_ids,keyid_to_key, topics):
    for topic in topics:
        key = keyid_to_key[topic]
        print(f'For topic {key}')
        for entity in key_to_ids[key]:
            id, prob = entity
            print("Doc: ",docs[id])
        print('---------------------------------')



In [ ]:
print_if_above_threshold(books.global_paragraphs, key_to_docids)

Topic 107 is above threhsold, it contains the following documents: 
Length:  14
--------------------------------------------------------------------------------
Probability:  0.9827809
Doc:  First, the bad news: operating systems concepts are among the most complex in computerscience. A modern, general-purpose operating system can exceed 50 million lines of code,or in other words, more than a thousand times longer than this textbook. New operatingsystems are being written all the time: if you use an e-book reader , tablet, or smartphone,an operating system is managing your device. Given this inherent complexity , we limit ourfocus to the essential concepts that every computer scientist should know 
Probability:  0.981312
Doc:  Making it easy to port applications from existing systems to a new operating system canhelp a new system become established; conversely , designing an operating system APIthat makes it dif ficult to port applications away from the operating system can help preven

In [ ]:
lda_index = Faiss(list(key_to_embeddings.values()), models.encoder_models)

In [ ]:
normal_index = Faiss(books.global_doc_embeddings, models.encoder_models)

In [ ]:
question = "What are privileged instructions?"
D,I = lda_index.serve_query(question)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2722: UserWarning: Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.
  warnings.warn("Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.")


In [ ]:
print_docs(books.global_paragraphs, I[0])

Doc:  Instructions available in kernel mode, but not in user mode, are called privilegedinstructions . The operating system kernel must be able to execute these instructions to doits work — it needs to change privilege levels, adjust memory access, and disable andenable interrupts. If these instructions were available to applications, then a rogueapplication would in ef fect have the power of the operating system kernel
Doc:  2.11 Summary and Future DirectionsThe process concept — the ability to execute arbitrary user programs with restricted rights— has been remarkably successful. With the exception of devices that run only a singleapplication at a time (such as embedded systems and game consoles), every commerciallysuccessful operating system provides process isolation
Doc:  The instructions to mask and unmask interrupts must be privileged; otherwise, user code
Doc:  Privileged instructions: instructions available only in kernel mode
Doc:  Other instructions are also limited to use b

In [ ]:
D, I = normal_index.serve_query(question)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2722: UserWarning: Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.
  warnings.warn("Though `pad_to_max_length` = `True`, it is ignored because `padding`=`True`.")


In [ ]:
def print_docs(docs, retr):
    for id in retr:
        print("Doc: ",docs[id])
    return

In [ ]:
D

array([[0.73964655, 0.7552498 , 0.76811624, 0.7718016 , 0.7767606 ]],
      dtype=float32)

In [ ]:
I[0]

array([107,  43,  29,  66,  49])

In [ ]:
print_retrieved(books.global_paragraphs, key_to_docids, keyid_to_key, I[0])

For topic 40
Doc:  Every time the processor fetches an instruction, it checks the address of the programcounter to see if it is between the base and the bound registers. If so, the instruction fetchis allowed to proceed; otherwise, the hardware raises an exception, suspending theprogram and transferring control back to the operating system kernel. Although it mightseem extravagant to perform two extra comparisons for each instruction, memoryprotection is worth the cost. In fact, we will discuss much more sophisticated and“extravagant" memory protection schemes in Chapter 8.
Doc:  a. Explain where in the operating system this instruction would be used
Doc:  b. Explain what happens if an application program executes this instruction
Doc:  13. Suppose you have to implement an operating system on hardware that supportsinterrupts and exceptions but does not have a trap instruction. Can you devise asatisfactory substitute for traps using interrupts and/or exceptions? If so, explain how 
Doc:

In [ ]:
class Book:
    def __init__(self, book_paths, device=DEVICE):
        self.models = Models()
        self.book_paths = book_paths

        #some global identifiers
        self.global_paragraphs = None
        self.global_doc_embeddings = None
        self.pobj = None
        self.num_paragraphs = 0

        #read book
        self.books = [PdfParagraphExtractor(book_path, self.models.encoder_model) for book_path in book_paths]

        self.bookid_to_pid = defaultdict(list)
        self.pid_to_bookid = defaultdict(int)
        self.set_global_info()

        ##Hypergraph Models
        self.kw_model = KeyBertModel(self.global_paragraphs, self.global_doc_embeddings)
        self.lda = LDA(self.global_paragraphs, self.global_doc_embeddings)

        ##hypergraph maps
        #keyword
        self.keyword_hg_embeddings = np.vstack(list(self.kw_model.key_map.values()))
        self.keyword_list = list(self.kw_model.key_map.keys())
        self.key_to_ids = self.kw_model.key_to_ids

        #topics
        self.topic_hg_embeddings = self.lda.arr_embeddings
        self.topic_dictionary = self.lda.topic_dictionary
        self.topic_list = self.lda.arr_topics

        ##indexes
        self.topic_index = Faiss(self.topic_hg_embeddings, self.query_encoder_model)
        self.keyword_index = Faiss(self.keyword_hg_embeddings, self.query_encoder_model)
        self.vector_index = Faiss(self.global_doc_embeddings, self.query_encoder_model)

    def set_global_info(self):
        all_ps = []
        all_objs = []
        all_doc_embeddings = []

        for bookid, book in enumerate(self.books):
            for pid, paragraph in enumerate(book.paragraphs):
                all_ps.append(paragraph)
                all_doc_embeddings.append(book.doc_embeddings[pid])

                self.bookid_to_pid[bookid].append(pid)
                self.pid_to_bookid[pid] = bookid

        self.num_paragraphs = len(all_ps)
        self.global_paragraphs = all_ps
        self.global_doc_embeddings = all_doc_embeddings

        for id in range(self.num_paragraphs):
            all_objs.append(Paragraph(self.pid_to_bookid[id], id, all_ps[id], all_doc_embeddings[id]))
        self.pobj = all_objs

        return

    def retrieve(self, query: str):
        query = query.replace('\n', '').replace('\t', '')

        topic_scores, topic_res = self.topic_index.serve_query(query)
        key_scores, key_res = self.keyword_index.serve_query(query)

        para_scores, para_res = self.vector_index.serve_query(query)

        topic_res = topic_res[0]
        key_res = key_res[0]
        para_res = para_res[0]

        topic_docs = []
        key_docs = []
        para = []

        for topic_id in topic_res:

            topic = self.topic_list[topic_id]
            pids = self.topic_dictionary[topic]

            ps = [self.global_paragraphs[pid[0]] for pid in pids]
            doc = ' '.join(ps)
            topic_docs.append(doc)

        for key_id in key_res:
            key = self.keyword_list[key_id]
            pids = self.key_to_ids[key]

            ps =  [self.global_paragraphs[pid] for pid in pids]
            doc = ' '.join(ps)
            key_docs.append(doc)

        for para_id in para_res:
            para.append(self.global_paragraphs[para_id])

        return (topic_docs, key_docs, para)


    def encode_passages(self, batch_text_passages, tokenizer, max_length):
        p = tokenizer(
            batch_text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        passage_ids = p['input_ids'][None]
        passage_masks = p['attention_mask'][None]

        passage_ids = np.transpose(passage_ids, (1, 0, 2))
        passage_masks = np.transpose(passage_masks, (1, 0, 2))

        return passage_ids, passage_masks.bool()

    def append_question(self, question,docs):
        return [question + " " + t for t in docs]

    def generate(self, docs, question):
        text_passages = [self.append_question(question,passage) for passage in docs]

        context_ids,context_mask=self.encode_passages(text_passages,self.decoder_tokenizer,1000)

        ans=None
        with torch.no_grad():
            outputs = self.decoder_model.generate(
                    input_ids=context_ids.to(device),
                    attention_mask=context_mask.to(device),
                    max_length=100)
            torch.cuda.empty_cache()
            for k, o in enumerate(outputs):
                ans = self.decoder_tokenizer.decode(o, skip_special_tokens=True)
        return ans












In [ ]:
pdf_paths = ['/content/Distributed Systems - 3rd Edition (Maarten van Steen, Andrew S. Tanenbaum) (z-lib.org).pdf']

In [ ]:
books = Book(pdf_paths)

Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base and are newly initialized: ['bert_model.embeddings.LayerNorm.bias', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.word_embeddings.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encoder.layer.0.attention.self.value.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.0.i

torch.Size([32, 191])
torch.Size([32, 230])
torch.Size([32, 230])
torch.Size([32, 179])
torch.Size([32, 159])
torch.Size([32, 276])
torch.Size([32, 214])
torch.Size([32, 188])
torch.Size([32, 205])
torch.Size([32, 198])
torch.Size([32, 285])
torch.Size([32, 311])
torch.Size([32, 286])
torch.Size([32, 199])
torch.Size([32, 264])
torch.Size([32, 242])
torch.Size([32, 313])
torch.Size([32, 260])
torch.Size([32, 246])
torch.Size([32, 172])
torch.Size([32, 328])
torch.Size([32, 195])
torch.Size([32, 222])
torch.Size([32, 272])
torch.Size([32, 225])
torch.Size([32, 187])
torch.Size([32, 222])
torch.Size([32, 288])
torch.Size([32, 262])
torch.Size([32, 211])
torch.Size([32, 245])
torch.Size([32, 165])
torch.Size([32, 201])
torch.Size([32, 197])
torch.Size([32, 151])
torch.Size([32, 268])
torch.Size([32, 166])
torch.Size([32, 241])
torch.Size([32, 218])
torch.Size([32, 189])
torch.Size([32, 188])
torch.Size([32, 154])
torch.Size([32, 234])
torch.Size([32, 170])
torch.Size([32, 155])
torch.Size

KeyboardInterrupt: 

In [ ]:
# books.kw_model.set_keyword_map(books.global_doc_embeddings, 0)

In [ ]:
# books.kw_model.key_to_ids

# Exp

In [ ]:
def generate(obj, docs, question):
    text_passages = obj.append_question(question,docs)

    context_ids,context_mask=obj.encode_passages(text_passages,obj.decoder_tokenizer,1000)
    print("SHape of context ids: ",context_ids.shape)
    ans=None
    with torch.no_grad():
        outputs = obj.decoder_model.generate(
                input_ids=context_ids.to(device),
                attention_mask=context_mask.to(device),
                max_length=500)
        torch.cuda.empty_cache()
        anss = []

        generated_answers = [obj.decoder_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return generated_answers

In [ ]:
def retrieve(obj, query: str):
    query = query.replace('\n', '').replace('\t', '')

    topic_scores, topic_res = obj.topic_index.serve_query(query)
    key_scores, key_res = obj.keyword_index.serve_query(query)

    topic_res = topic_res[0]
    key_res = key_res[0]

    topic_docs = []
    key_docs = []

    for topic_id in topic_res:

        topic = books.topic_list[topic_id]
        pids = books.topic_dictionary[topic]

        ps = [books.global_paragraphs[pid[0]] for pid in pids]
        doc = ' '.join(ps)
        topic_docs.append(doc)

    for key_id in key_res:
        key = books.keyword_list[key_id]
        pids = books.key_to_ids[key]

        ps =  [books.global_paragraphs[pid] for pid in pids]
        doc = ' '.join(ps)
        key_docs.append(doc)
    all_docs = key_docs+topic_docs
    print(all_docs)
    print(type(all_docs))
    return all_docs
    # ans = obj.generate(all_docs, query)
    # return {
    #     'topic': topic_docs,
    #     'key': key_docs,
    #     'ans': ans
    # }

In [ ]:
def encode_passages(batch_text_passages, tokenizer, max_length):
    p = tokenizer(
        batch_text_passages,
        max_length=max_length,
        pad_to_max_length=True,
        return_tensors='pt',
        truncation=True
    )
    passage_ids = p['input_ids'][None]
    passage_masks = p['attention_mask'][None]

    return passage_ids, passage_masks.bool()

In [ ]:
question = '''What do you mean by Kernel?'''

In [ ]:
d = books.retrieve(question)

['Exercises', 'Creating and managing processes', 'Allocate memory for the process', 'wait(processID)Pause until the child process has exited', 'Server clusters.  For fault tolerance, scale, and responsiveness, web sites areincreasingly implemented on distributed clusters of computers housed in one or more', 'Throughout this book, we will present various approaches to improving operating systemreliability and availability . In many cases, the abstractions may seem at first glance overlyrigid and formulaic. It is important to realize this is done on purpose! Only preciseabstractions provide a basis for constructing reliable and available systems', 'wait(processID)Pause until the child process has exited', 'This starts the handler software', 'Interrupt masking prevents interrupts from being delivered at inopportune times', 'This starts the handler software', 'Suppose further that everyone in the world were completely honest and trustworthy . Inother words, we need not consider fault isola

In [ ]:
generate(books, d, question)

In [ ]:
d

In [ ]:
# books.topic_index.serve_query(question)[1][0]

array([ 33,  96,  37,  12,  19,  61,  97,  83,  27, 112])

In [ ]:
# d = retrieve(books, question)

['Thrashing', 'Databases', 'Detecting Deadlock', 'Compacttranslationtables', 'FIFORef', 'sync', 'Returnvalues', 'Pipelining', 'Exercises', 'Whichpagesshouldbeevicted', 'This brings us to Solution 3 Solution 3.  Solution 2 was safe because a thread would avoid buying milk if there wereany chance that the other thread might  buy milk. For Solution 3, we ensure that at leastone of the threads determines whether the other thread has bought milk or not beforedeciding whether or not to buy . In particular , we do the following:  Path A noteA = 1;         // leave note A   while (noteB==1) { // wait for no note B       ;              // spin   }  if (milk==0) {     // if no milk M       milk++;        // buy milk   }  noteA = 0;         // remove note A   Path B noteB = 1;          // leave note B   if (noteA==0) {     // if no note A       if (milk==0) {  // if no milk           milk++;     // buy milk       }               //   }                   //   noteB = 0;          // remove note B  

In [ ]:
d

['Exercises',
 'Creating and managing processes',
 'Allocate memory for the process',
 'wait(processID)Pause until the child process has exited',
 'Server clusters.  For fault tolerance, scale, and responsiveness, web sites areincreasingly implemented on distributed clusters of computers housed in one or more',
 'Throughout this book, we will present various approaches to improving operating systemreliability and availability . In many cases, the abstractions may seem at first glance overlyrigid and formulaic. It is important to realize this is done on purpose! Only preciseabstractions provide a basis for constructing reliable and available systems',
 'wait(processID)Pause until the child process has exited',
 'This starts the handler software',
 'Interrupt masking prevents interrupts from being delivered at inopportune times',
 'This starts the handler software',
 'If not, explain why  Explain (in English) how you would use UNIX system calls to automate testing ofsubmitted homework as

In [ ]:
# books.encode_passages(['This',''])

In [ ]:
generate(books, d, question)

SHape of context ids:  torch.Size([1, 20, 1000])


['a kernel with a kernel with a kernel with a kernel. What do you mean by Kernel? What is the processor f the processoras well as in thread systems for creating a new thread, treat all users equally??b... But an operating system must.. then exits the process..  or crashed. is terminated. with']

In [ ]:
generate(books, d, question)

'a kernel with a kernel with a kernel with a kernel. What do you mean by Kernel? What is the processor f the processoras well as in thread systems for creating a new thread, treat all users equally??b... But an operating system must.. then exits the process..'

In [ ]:
context_ids,context_mask = encode_passages(d, books.decoder_tokenizer, 1000)

In [ ]:
context_ids.shape

torch.Size([1, 20, 1000])

In [ ]:
for i in d:
    print(type(i))
    print(len(i))

<class 'str'>
24
<class 'str'>
12
<class 'str'>
20
<class 'str'>
6
<class 'str'>
12
<class 'str'>
15
<class 'str'>
22
<class 'str'>
10
<class 'str'>
31
<class 'str'>
21
<class 'str'>
326
<class 'str'>
664
<class 'str'>
1402
<class 'str'>
1030
<class 'str'>
1083
<class 'str'>
1691
<class 'str'>
1410
<class 'str'>
2058
<class 'str'>
1278
<class 'str'>
1040


In [ ]:
books.generate(d, question)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 24 but got size 12 for tensor number 1 in the list.

In [ ]:
books.books[0].pdf_contents

{'Contents': 5,
 'Preface': 8,
 '1 Introduction': 17,
 '2 The Kernel Abstraction': 47,
 '3 The Programming Interface': 95,
 'References': 125,
 'Glossary': 136,
 'About the Authors': 163}

In [ ]:
js['keyword'][1]

array([[1422, 1699, 1207,  669,  424, 1714,  238,  580, 1384,  790]])

In [ ]:
for i in js['topic'][1][0]:
    print(i)
    topic_id = books.topic_list[i]
    pids = books.topic_dictionary[topic_id]
    for pid in pids:
        print(books.global_paragraphs[pid[0]])

In [ ]:
for i in js['keyword'][1][0]:
    print(i)
    key = books.keyword_list[i]
    pids = books.key_to_ids[key]
    for pid in pids:
        print(books.global_paragraphs[pid])

In [ ]:
books.kw_model.key_to_ids

In [ ]:
books.kw_model.extract(books.global_paragraphs, books.global_doc_embeddings)

In [ ]:
books.topic_dictionary

In [ ]:
books.key_to_ids

In [ ]:
# arr_topics = list(lda.hyperedge_embeddings.keys())

In [ ]:
# arr_embeddings = list(lda.hyperedge_embeddings.values())

In [ ]:
# arr_embeddings = np.vstack(arr_embeddings)